In [3]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.distributions import Categorical
from collections import namedtuple
from torch import optim
import numpy as np
import numpy as np
import math
import gym


In [4]:
class PolicyNetwork(nn.Module):
    def __init__(self, state_space, n_actions):
        super(PolicyNetwork, self).__init__()
        self.base = nn.Linear(state_space, 256)
        self.actions = nn.Linear(256, n_actions)
        self.value = nn.Linear(256, 1)
        self.rewards = []
        self.action_pairs = [] #[(LOG_PROB, CRITIC_VALUE)]
    def forward(self, x):
        x = F.relu(self.base(x))
        a = F.softmax(self.actions(x), dim=-1)
        v = self.value(x)
        return a,v
    def select_action(self, state):
        a, v = self.forward(state)
        m = Categorical(a)
        action = m.sample()
        self.action_pairs.append((m.log_prob(action), v))
        return action.item()
    def train(self, OPTIM, gamma):
        ##CONSTRUCT SAMPLED VALUES##
        rewards = []
        R = 0
        for r in self.rewards[::-1]:
            R = r + gamma * R
            rewards.insert(0, R)
        ##NORMALIZE SAMPLED STATE VALUES##
        rewards = torch.tensor(rewards, requires_grad=False)
        rewards = (rewards-rewards.mean())/(rewards.std() + 1e-4)
        ##GET ACTOR AND CRITIC LOSS##
        actor_loss = torch.tensor([0], dtype=torch.float32)
        critic_loss = torch.tensor([0], dtype=torch.float32)
        for (log_prob, val), R in zip(self.action_pairs, rewards):
            advantage = R.item() - val.item()
            actor_loss += -log_prob*advantage
            critic_loss += F.smooth_l1_loss(val, torch.tensor([[R]]))
        total_loss = actor_loss + critic_loss
        ##OPTIMIZE##
        OPTIM.zero_grad()
        total_loss.backward()
        OPTIM.step()
        ##CLEAR MEMORY##
        del self.rewards[:]
        del self.action_pairs[:]
        

In [5]:
ac = PolicyNetwork(4, 2)
ac.select_action(torch.randn(1,4))

1

In [6]:
EPOCHS = 1000
ac = PolicyNetwork(4,2)
env = gym.make("CartPole-v1", render_mode="human")
OPTIM = optim.Adam(ac.parameters(), 3e-3)
GAMMA = 0.99

In [ ]:
TRAIN_REWARDS = []
moving_average = 0
solved = False

for epoch in range(EPOCHS):
    total_reward = 0
    state = env.reset()
    state = torch.Tensor(state[0]).unsqueeze(0)
    while (True):
        action = ac.select_action(state)

        next_state, reward, done, _, _ = env.step(action)
        next_state = torch.Tensor(next_state)
        ac.rewards.append(reward)
        total_reward += reward

        state = next_state.unsqueeze(0)
        if(total_reward > 999):
            print("SOLVED")
            TRAIN_REWARDS.append(total_reward)
            solved = True
            break
        
        if done:
            ac.train(OPTIM, GAMMA)
            TRAIN_REWARDS.append(total_reward)
            moving_average = moving_average*(0.8) + (0.2)*total_reward
            if(epoch % 100 == 0):
                print(f"{epoch} Epoch total moving average {moving_average}")
                
            break
    if solved:
        break

env.close()


/home/lk3ond/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


0 Epoch total moving average 5.2


In [ ]:
import matplotlib.pyplot as plt
plt.plot(TRAIN_REWARDS)
plt.ylabel('rewards')
plt.xlabel('episodes')
plt.title("Results of Actor Critic")
plt.show()